In [4]:
#필요라이브러리 설치
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print('hello')

# 버전 확인
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")

hello
pandas 버전: 2.2.3
numpy 버전: 2.4.0


In [5]:
#현재 작업 경로 확인
import os
print(os.getcwd())

#파일 불러오기
df = pd.read_csv('rd/union1.csv', encoding='utf-8')
print("cp949로 로드 성공!")

c:\Users\user\Desktop\STA_track\bds\budongsan3_project\ysy
cp949로 로드 성공!


In [6]:
#데이터 살펴보기

df.head()
df.info()
df.describe()
df.shape
df.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29712 entries, 0 to 29711
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구코드   29712 non-null  int64  
 1   자치구명    29712 non-null  object 
 2   법정동코드   29712 non-null  int64  
 3   법정동명    29712 non-null  object 
 4   본번      29708 non-null  float64
 5   부번      29708 non-null  float64
 6   건물명     29712 non-null  object 
 7   계약일     29712 non-null  int64  
 8   물건금액    29712 non-null  int64  
 9   건물면적    29712 non-null  float64
 10  층       29712 non-null  int64  
 11  건축년도    29712 non-null  int64  
 12  건물용도    29712 non-null  object 
 13  신고구분    29712 non-null  object 
dtypes: float64(3), int64(6), object(5)
memory usage: 3.2+ MB


['자치구코드',
 '자치구명',
 '법정동코드',
 '법정동명',
 '본번',
 '부번',
 '건물명',
 '계약일',
 '물건금액',
 '건물면적',
 '층',
 '건축년도',
 '건물용도',
 '신고구분']

In [7]:
#계약일을 datetime형식으로 변환
df['계약일'] = pd.to_datetime(df['계약일'], format= '%Y%m%d')
df['물건금액'] = round(df['물건금액'], 0)

print(df['계약일'].head())
print(df['물건금액'].head())

0   2023-12-31
1   2023-12-31
2   2023-12-30
3   2023-12-30
4   2023-12-30
Name: 계약일, dtype: datetime64[ns]
0    182000
1    162500
2     25700
3    210000
4    323000
Name: 물건금액, dtype: int64


In [8]:

#자치구-법정동 기준
trend_detail = df.groupby(['계약일','자치구명','법정동명'])['물건금액'].mean().round(0).unstack(level=['자치구명','법정동명']).sort_index(axis=1)

#자치구별 기준
trend_gu = trend_detail.groupby(level='자치구명', axis=1).mean().round(0)

#전체 강남3구 기준
trend_total = df.groupby(['계약일'])['물건금액'].mean().round(0)  
overall_avg = trend_total.mean().mean()

print(f"전체 강남3구 기준 평균거래금액: {overall_avg:,.2f}만 원")

print(trend_detail.head())
print(trend_total.head())

전체 강남3구 기준 평균거래금액: 218,501.87만 원
자치구명             강남구                                             ... 송파구      \
법정동명             개포동 논현동       대치동 도곡동 삼성동 세곡동 수서동 신사동 압구정동 역삼동  ... 문정동 방이동   
계약일                                                              ...           
2023-01-01       NaN NaN       NaN NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-02  166900.0 NaN       NaN NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-03  180625.0 NaN  186000.0 NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-04       NaN NaN       NaN NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-05  204000.0 NaN  300000.0 NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   

자치구명                                                    
법정동명       삼전동 석촌동 송파동       신천동 오금동       잠실동 장지동 풍납동  
계약일                                                     
2023-01-01 NaN NaN NaN       NaN NaN       NaN NaN NaN  
2023-01-02 NaN NaN NaN       NaN NaN       NaN NaN NaN  
2023-01-03 NaN NaN NaN  172000.0 NaN  217

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26840\2182708038.py:5: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  trend_gu = trend_detail.groupby(level='자치구명', axis=1).mean().round(0)


In [9]:
# 인덱스를 datetime으로 변경⭐⭐⭐ 
# groupby하면서 '계약일'이 인덱스가 되면서 datetime이 풀렸을 가능성이 큼
trend_detail.index = pd.to_datetime(trend_detail.index)
trend_gu.index = pd.to_datetime(trend_gu.index)
trend_total.index = pd.to_datetime(trend_total.index)

#구-동 일별 데이터 ==> 월별로 resampling하기
trend_detail_monthly = trend_detail.resample('MS').mean()

# 구별 데이터를 일별 ==> 월별로 resampling하기
trend_gu_monthly = trend_gu.resample('MS').mean()

# 강남3구 데이터를 일별 ==> 월별로 resampling하기
trend_total_monthly = trend_total.resample('MS').mean()

print(trend_detail_monthly.head())
print(trend_gu_monthly.head())
print(trend_total_monthly.head())

print(f"시구별 데이터 행 개수: {len(trend_detail_monthly)}")
print(f"구별 데이터 행 개수: {len(trend_gu_monthly)}")
print(f"강남3구 전체 평균 행 개수: {len(trend_total_monthly)}")

자치구명                  강남구                                               \
법정동명                  개포동            논현동            대치동            도곡동   
계약일                                                                      
2023-01-01  185056.050000            NaN  250450.000000  233100.000000   
2023-02-01  164280.913043  126671.428571  212597.368421  182848.571429   
2023-03-01  153964.600000   86516.666667  243302.062500  205575.000000   
2023-04-01  199624.421053  126125.000000  258393.117647  254994.700000   
2023-05-01  184419.350000  179471.428571  246278.652174  265224.150000   

자치구명                                                                         \
법정동명                  삼성동       세곡동            수서동       신사동           압구정동   
계약일                                                                           
2023-01-01   35250.000000       NaN  134714.285714   62500.0  304250.000000   
2023-02-01  176750.000000  137250.0  111549.272727       NaN  365214.285714   
2023-03-01  

In [17]:
import pandas as pd
import plotly.graph_objects as go

# 1. 데이터 집계 (기존 로직 유지)
df_2025 = df[df['계약일'].dt.year == 2025].copy()
df_2025['월_숫자'] = df_2025['계약일'].dt.month
df_2025['월'] = df_2025['월_숫자'].astype(str) + "월"

monthly_gu = df_2025.groupby(['월_숫자', '월', '자치구명'])['물건금액'].mean().reset_index()
monthly_total = df_2025.groupby(['월_숫자', '월'])['물건금액'].mean().reset_index()
monthly_total['자치구명'] = '강남3구 전체'
trend_df = pd.concat([monthly_gu, monthly_total]).sort_values('월_숫자')

# [기능 분리] 1. 레이블용 상세 포맷 (억 + 만원)
def format_detailed_won(val):
    uk = int(val // 10000)
    man = int(val % 10000)
    if uk > 0:
        return f"{uk}억 {man:,.0f}만"
    else:
        return f"{man:,.0f}만"

# [기능 분리] 2. Y축 눈금용 포맷 (억 단위만)
def format_only_uk(val):
    return f"{int(val // 10000)}억"

# 2. 선 그래프 생성
fig = go.Figure()

colors = {'강남구': 'royalblue', '서초구': 'darkorange', '송파구': 'forestgreen', '강남3구 전체': 'red'}
widths = {'강남구': 2.0, '서초구': 2.0, '송파구': 2.0, '강남3구 전체': 3.75}

for name in ['강남구', '서초구', '송파구', '강남3구 전체']:
    target_data = trend_df[trend_df['자치구명'] == name]
    
    fig.add_trace(go.Scatter(
        x=target_data['월'],
        y=target_data['물건금액'],
        name=name,
        mode='lines+markers',
        line=dict(color=colors[name], width=widths[name]),
        marker=dict(size=8),
        hoverinfo='name+x+y'
    ))

    # 최고/최저점 데이터 레이블 (세밀하게 표시)
    max_row = target_data.loc[target_data['물건금액'].idxmax()]
    min_row = target_data.loc[target_data['물건금액'].idxmin()]

    for row, pos in [(max_row, 'top'), (min_row, 'bottom')]:
        fig.add_annotation(
            x=row['월'],
            y=row['물건금액'],
            text=format_detailed_won(row['물건금액']), # 상세 단위 사용
            showarrow=False,
            yshift=12 if pos == 'top' else -12,
            font=dict(color=colors[name], size=10)
        )

# 3. Y축 눈금 설정 (억 단위로 표시)
max_val = trend_df['물건금액'].max()
min_val = 100000 # 10억
tick_vals = [i for i in range(min_val, int(max_val) + 50000, 50000)]
tick_texts = [format_only_uk(v) for v in tick_vals] # 억 단위만 사용

# 4. 레이아웃 설정
fig.update_layout(
    title=dict(
        text='<b>2025년 강남 3구 지역별/전체 월별 실거래가 추이</b>',
        x=0.5, y=0.88, font=dict(size=22)
    ),
    template='plotly_white',
    width=1100, height=750,
    margin=dict(l=100, r=50, t=140, b=100),
    xaxis=dict(title=None, showgrid=False),
    yaxis=dict(
        title="평균 실거래가 (Average Transaction Price)",
        tickmode='array',
        tickvals=tick_vals,
        ticktext=tick_texts, # Y축은 깔끔하게 억 단위
        showgrid=True,
        gridcolor='rgba(200, 200, 200, 0.3)',
        gridwidth=1,
        griddash='dot',
        range=[min_val, max_val * 1.15]
    ),
    legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="center", x=0.5)
)

fig.show()